In [1]:
import pickle
import numpy as np
import tensorflow as tf
file = 'data_batch_1'
data_size = 10000

In [2]:
def generate_one_hot(labels):
    one_hot = []
    mold = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for label in labels:
        temp = mold[:]
        temp[label] = 1
        one_hot.append(temp)
    return one_hot

In [3]:
def get_batches(pool):
    batches = []
    len = pool.__len__()
    lo = 0
    hi = 1000
    for b in range(10):
        batches.append(pool[lo:hi])
        hi += 1000
        lo += 1000
    return batches

CIFAR-10  data is loaded, only one batch is being used right now.

In [4]:
with open(file, 'rb') as f:
    dict = pickle.load(f, encoding='bytes')
images = dict[b'data']

labels = dict[b'labels']

test_labels = labels[2000:2100]
labels = labels[:data_size]

labels = generate_one_hot(labels)
test_labels = generate_one_hot(test_labels)

labels = np.array(labels)
test_labels = np.array(test_labels)

labels = np.reshape(labels, (data_size, 10))
test_labels = np.reshape(test_labels, (100, 10))


test_images = images[2000:2100]
images = images[:data_size]

images.__len__()

ib = get_batches(images)
lb = get_batches(labels)

print(images[1000], ib[1][0], labels[1000], lb[1][0])

[242 242 244 ..., 137 137 135] [242 242 244 ..., 137 137 135] [0 0 0 0 0 0 0 0 0 1] [0 0 0 0 0 0 0 0 0 1]


Constants and parameters are set up

In [5]:
x = tf.placeholder('float', [None, 3072])
y = tf.placeholder('float', [None, 10])

Functions for actually performing convolutions and pooling

In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding='SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [7]:
def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,3,32])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([8*8*64, 1024])),
               'out':tf.Variable(tf.random_normal([1024, 10]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([10]))}

    x = tf.reshape(x, shape=[-1, 32, 32, 3])

    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    fc = tf.reshape(conv2 ,[-1, 8*8*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

In [13]:
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    print(prediction, y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 100
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        epoch_error = 0
        for epoch in range(hm_epochs):
            for e_image, e_label in zip(ib, lb):
                _, c = sess.run([optimizer, cost], feed_dict={x:e_image, y:e_label})
                epoch_error += c
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',c)
                
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:test_images, y:test_labels}))


In [14]:
train_neural_network(x)

Tensor("add_7:0", shape=(?, 10), dtype=float32) Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 100 loss: 5.55292e+07
Epoch 0 completed out of 100 loss: 4.00112e+07
Epoch 0 completed out of 100 loss: 3.16735e+07
Epoch 0 completed out of 100 loss: 2.73446e+07
Epoch 0 completed out of 100 loss: 2.30033e+07
Epoch 0 completed out of 100 loss: 2.11136e+07
Epoch 0 completed out of 100 loss: 1.96153e+07
Epoch 0 completed out of 100 loss: 2.03299e+07
Epoch 0 completed out of 100 loss: 1.93786e+07
Epoch 0 completed out of 100 loss: 1.76478e+07
Epoch 1 completed out of 100 loss: 1.55273e+07
Epoch 1 completed out of 100 loss: 1.42945e+07
Epoch 1 completed out of 100 loss: 1.25255e+07
Epoch 1 completed out of 100 loss: 1.2674e+07
Epoch 1 completed out of 100 loss: 1.16629e+07
Epoch 1 completed out of 100 loss: 1.13581e+07
Epoch 1 completed out of 100 loss: 1.09548e+07
Epoch 1 completed out of

KeyboardInterrupt: 